In [1]:
import sqlite3

import pandas as pd

#Connection à la base de donnée
connection = sqlite3.connect('babies.db')

#connection à la base de donnée
cursor = connection.cursor()

#Création et remplissage des tables de notre base de données 
Les requêtes de la création manuelle sont disponibles dans le rapport associé.

Etant donné que la base de donnée est open-data et que n'anonymat des individus est assuré nous n'avons pas pu avoir les identifiants réels des individus donc nous avons utilisé les numéros de ligne à la place pour garder des résultats cohérents avec la partie python

# Importation des données pour remplir les tables

In [11]:
bebe =pd.read_csv("bebe.csv")
mere = pd.read_csv("mere.csv")
pere = pd.read_csv("pere.csv")
zone = pd.read_csv("zone.csv")

bebe.to_sql("bebe",connection,if_exists="replace")
mere.to_sql("mere",connection,if_exists="replace")
pere.to_sql("pere",connection,if_exists="replace")
zone.to_sql("zone",connection,if_exists="replace")

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:2615: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


#Les requêtes de filtrage

In [4]:
#Tous le bébés avec un poids plus petit que 2.5 kg
req ="""
SELECT * 
FROM bebe
WHERE poids_bebe <2.5
"""
pd.read_sql_query(req,connection)

,index,id_bebe,ordre_naissance,previs,wic,medoc_fertilite,born_hopital,poids_bebe,dplural,duree_grossesse,sex,pere_race,mere_race,id_mere,id_pere
0,3,4,6,15,1.0,X,1,2.125,1.0,37.0,1.0,2,10,4,4
1,20,22,1,13,0.0,X,1,1.290,1.0,31.0,0.0,3,10,22,22
2,62,65,3,4,1.0,X,1,1.190,1.0,28.0,0.0,1,10,65,65
3,70,75,3,0,0.0,X,1,2.155,1.0,39.0,0.0,9,61,75,75
4,80,85,3,14,0.0,X,1,1.660,1.0,32.0,1.0,4,40,85,85
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155129,2053397,2425135,4,11,0.0,X,1,2.415,2.0,39.0,0.0,1,10,2425135,2425135
155130,2053429,2425172,2,15,0.0,X,1,2.410,1.0,39.0,0.0,1,10,2425172,2425172
155131,2053431,2425174,1,12,0.0,X,1,2.495,2.0,34.0,1.0,1,10,2425174,2425174
155132,2053437,2425180,1,12,0.0,X,1,2.250,2.0,34.0,1.0,1,10,2425180,2425180


In [5]:
#Nombre de jumeaux dont la mère n'a pas suivi de consultation prénatale pendant leur grossesse et leur poids moyen
req = """
SELECT COUNT(*) as nombre_sans_visite , AVG(poids_bebe) as poids_moyen
FROM bebe
WHERE previs =2 AND dplural = 2 
"""
pd.read_sql_query(req,connection)

,nombre_sans_visite,poids_moyen
0,455,1.870523


il ya 455 enfants dont la mère n'a pas suivi de consultation prénatale pendant toute la durée de leur grossesse

# Requete utilisant group by

In [7]:
#Liste poids moyen des bébé par sexe
req ="""
SELECT sex, AVG(poids_bebe) as poids_moyen
FROM bebe
GROUP BY sex
"""
pd.read_sql_query(req,connection)

,sex,poids_moyen
0,0.0,3.225643
1,1.0,3.340414


Le poids moyen des enfants de sexe masculin est 3,3 kg et celui des enfants de sexe masculin est 3,3 Kg

#Requêtes avec jointures

In [8]:
#Liste bébés dont les mères sont obèses
#Une personne obèse a un imc de plus de 35
req = """
SELECT * 
FROM bebe
INNER JOIN mere ON bebe.id_mere =  mere.id_mere
WHERE imc >= 35 
"""
pd.read_sql_query(req,connection)

,index,id_bebe,ordre_naissance,previs,wic,medoc_fertilite,born_hopital,poids_bebe,dplural,duree_grossesse,sex,pere_race,mere_race,id_mere,id_pere,index,id_mere,age_mere,nbre_cigarette_daily,imc,niv_educ_mere,mere_fumeuse,mere_race
0,3,4,6,15,1.0,X,1,2.125,1.0,37.0,1.0,2,10,4,4,3,4,37,0,35.5,4.0,0.0,10
1,24,26,1,13,1.0,X,1,3.425,1.0,40.0,1.0,6,10,26,26,24,26,29,1,39.0,4.0,1.0,10
2,45,47,8,14,0.0,X,1,3.770,1.0,37.0,0.0,1,61,47,47,45,47,26,0,35.5,3.0,0.0,61
3,50,53,7,99,0.0,X,1,3.742,1.0,39.0,1.0,1,61,53,53,50,53,29,0,39.9,3.0,0.0,61
4,53,56,1,13,0.0,X,1,3.590,1.0,38.0,0.0,1,10,56,56,53,56,24,0,42.0,5.0,0.0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256971,2053494,2425244,2,14,0.0,X,1,3.470,1.0,37.0,0.0,2,20,2425244,2425244,2053494,2425244,30,0,39.9,4.0,0.0,20
256972,2053500,2425252,2,4,0.0,X,1,3.640,1.0,39.0,0.0,1,10,2425252,2425252,2053500,2425252,35,0,40.9,6.0,0.0,10
256973,2053504,2425256,2,11,0.0,X,1,2.843,1.0,39.0,0.0,1,10,2425256,2425256,2053504,2425256,31,0,37.3,7.0,0.0,10
256974,2053511,2425264,3,99,0.0,X,1,2.670,1.0,36.0,1.0,1,10,2425264,2425264,2053511,2425264,34,10,41.6,6.0,1.0,10


#Requête avec HAVING

In [26]:
#Liste des zones ayant compté au moins 15000 enfants au cours de l'année
req ="""
SELECT code_zone,label , COUNT(*) as nombre_bebe
FROM bebe
INNER JOIN mere ON bebe.id_mere = mere.id_mere
INNER JOIN zone ON mere.code_zone = zone.code_zone
GROUP BY zone.code_zone
HAVING nombre_bebe >=15000
ORDER BY age_mere desc 
"""
pd.read_sql_query(req,connection)

,code_zone,label,nombre_bebe
0,61,Inconnu,107776
1,40,Asiatique (uniquement),252810
2,20,Noir (uniquement),549514
3,41,NHOPI (uniquement),56480
4,10,Blanc (uniquement),3093332
5,30,AIAN (uniquement),34640
